In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Implementación de modelos ANN para clasificación de objetos astrofísicos identificados por el SDSS

## ( Galaxias, Estrellas y Quasars)

Para dar solución a una máquina que logre clsificar las imágenes del Sloan Digital Sky Survey DR14, se realizaun modelo perceptron multicapa, se describe paso por paso cada una de sus aspectos más importantes.

In [ ]:
#import Data
import pandas as pd
data_f1=pd.read_csv('/kaggle/input/sloan-digital-sky-survey/Skyserver_SQL2_27_2018 6_51_39 PM.csv')
data_f1=data_f1.drop(['rerun'], axis=1)
data_f1.head()

In [ ]:
dim=display(data_f1.shape)

El target es la variable class, la cual es de tipo categorica y tiene como niveles:

In [ ]:
label = data_f1['class']   #Accediendo a una columna
label.head()

Con el fin de organizar nuestros datos establecemos que:

In [ ]:
data_f1.info()

In [ ]:
# Estadísticas de las columnas númericas
data_f1.describe()

In [ ]:
num = (data_f1.dtypes == 'float') | (data_f1.dtypes == 'int64')
num

In [ ]:
num_cols = []
for c in num.index:
    if num[c] == True:
        num_cols.append(c)                #.append  agrega un ítem al final de la lista, en este caso agrega c a num_cols
        
display(num_cols)

In [ ]:
data_f1_num = data_f1[num_cols]               #genera busqueda de 
data_f1_num.head()

# Estadísticas de las columnas númericas
data_f1_num.describe().T

In [ ]:
import matplotlib.pyplot as plt 
data_f1_num['dec'].hist()

In [ ]:
data_f1_num['u'].hist()

In [ ]:
data_f1_num['g'].hist()

In [ ]:
import seaborn as sns
sns.catplot(x="class", kind="count", palette="ch:.25", data=data_f1);

In [ ]:
sns.catplot(x="class", y="u", kind="box", data=data_f1);

Función lapply

La función lapply permite de alguna manera idnetificar los datos nulos del DataFrame

In [ ]:
data_f1_num.notnull().apply(pd.Series.value_counts) 

* Se puede observar que ninguna label presenta datos faltantes, ahora bien miremos si hay valores 0.
* En lo que corresponde a valores nulos es decir 0, se tiene quer en la variable redshift existen 19 ceros, quizas este puede ser un problema en el momento de generar el modelo.

In [ ]:
(data_f1 != 0).apply(pd.Series.value_counts)

In [ ]:
z=pd.concat([data_f1_num,label],axis=1)            #concatenando vas features con la variable respuesta.
z.head()
sns.pairplot(z)

In [ ]:
data_f1_num.iloc[0:5:, 0:5]
display(type(data_f1_num))

In [ ]:
#sns.pairplot(estrellas_num)
import seaborn as sb
sb.pairplot(data_f1_num,diag_kind='kde')

In [ ]:
correlation_dat = data_f1_num.corr()
correlation_dat.style.background_gradient(cmap='coolwarm', axis=None)

# Dividiendo los datos de entrenamiento y de validación

In [ ]:
X = data_f1.iloc[:, [1,2,3,4,5,6,7,13]]
Y = data_f1.iloc[:, [12]]
Y = Y.replace({"GALAXY":0, "STAR":1,"QSO":2})
print(X.head())


from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

y = to_categorical(Y)
X_std = StandardScaler().fit_transform(X)

In [ ]:
print(X_std,y)

train_test_split es una función en la selección del modelo de Sklearn para dividir matrices de datos en dos subconjuntos : datos para entrenamiento y datos para prueba. No obstante, train_test_split hará particiones aleatorias para los dos subconjuntos. Esta función toma como parámetros el conjunto de datos X_std y las etiquetas y.

train_size es el otro parámetro y establece el tamaño del conjunto de datos de entrenamiento, en este caso se usa 0.2 el cual hace referencia a 80% para test y 20% para prueba.

In [ ]:
X_trainT , X_test ,Y_trainT, Y_test = train_test_split(X_std , y , test_size=0.2)
X_train , X_test10 ,Y_train, Y_test10 = train_test_split(X_trainT , Y_trainT , test_size=0.125)

---
# Modelo perceptron multicapa 
---


Considerando que las redes neuronales son grupo de neuronas distribuidas en capas y conectadas por sus pesos (salidas con un valor asignado a la información que recibieron) se procede a describir el modelo ANN por medio de:

#### Primera capa

Capa densa de 5 neuronas y entrada de tamaño 8, con función de activación ReLu (Unidad rectificada lineal). Además un dropout de 0.45.

#### Segunda capa

Una capa densa con 256 neuronas y función de activación ReLu (Unidad rectificada lineal) y dropout de 0.45.

#### Tercera capa

Una capa densa con 3 neuronas y función de activación Softmax.
Caba aclarar que los lotes son de tamaño 128.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model

In [ ]:
input_size = 8
num_labels = 3
batch_size = 128
dropout = 0.45

from tensorflow.keras.regularizers import l2

model = Sequential()

model.add(Dense(5, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()

In [ ]:
plot_model(model, show_shapes=True)

---
# Compilación
---

Un algoritmo de deep learning demora tanto en entrenar (entre otras cosas) porque eventualmente encuentra muchos mínimos locales en la superficie de error.

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

---
# Entremiento del modelo
---

Aquí se caracterizará el checkpoint para poder guardar el modelo y después poder usarlo para hacer predicciones.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import backend as K

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.99):
            print("\nReached 0.99 accuracy so cancelling training!")
            self.model.stop_training = True

# crea una instancia de clase
accu_callback = myCallback()

In [ ]:

# Directory where the checkpoints will be saved
import os 
checkpoint_dir = './training_checkpoints_mnist_mlp'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "mnist_mlp_ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True, 
    monitor='val_accuracy', mode='max',
    save_best_only=True)

Se procede a realizar el entrenamiento del modelo ANN que fue caracterizado en lineas anteriores con epoch de tamaño 20 y un tamaño de validación de 0.2.

In [ ]:
epochs = 50
validation_size =0.2

history = model.fit(X_train, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_split=validation_size,
                    callbacks=[accu_callback,checkpoint_callback],
                    verbose=2)

---
Evaluación del modelo ANN
---

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

n_row = 1
n_col = 2
fig, ax = plt.subplots(n_row, n_col, sharex = False, sharey = False, figsize=(12,6))


ax[0].plot(epochs, acc, 'r', label='Training accuracy', color = 'green')
ax[0].plot(epochs, val_acc, 'b', label='Validation accuracy')
ax[0].legend(fontsize=12,loc=0)
ax[0].set_title('Training and Validation Accuracy',fontsize=16)
ax[0].set_ylabel('measure',fontsize=14)
ax[0].set_xlabel('epoch', fontsize = 14)
ax[0].set_xlim([1, len(acc)])

ax[1].plot(epochs, loss, 'r', label='Training Loss', color = 'green')
ax[1].plot(epochs, val_loss, 'b', label='Validation Loss')
ax[1].legend(fontsize=12)
ax[1].set_title('Training and Validation Loss',fontsize=16)
ax[1].set_ylabel('measure',fontsize=14)
ax[1].set_xlabel('epoch', fontsize = 14)
ax[1].set_xlim([1, len(acc)])


plt.show()